In [1]:
from huggingface_hub import hf_hub_download
import pandas as pd
import numpy as np
import dspy
import os


REPO_ID = "PatronusAI/HaluBench"
FILE_NAME = "data/test-00000-of-00001.parquet"
dataset_raw = pd.read_parquet(hf_hub_download(repo_id=REPO_ID, filename=FILE_NAME,repo_type="dataset"))

/home/tash01-admin/reasearch/myvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_raw["label"] = dataset_raw["label"].astype(str)
dataset_raw["source_ds"] = dataset_raw["source_ds"].astype(str)
#dataset_raw = dataset_raw[dataset_raw["source_ds"] == "FinanceBench" ]

dataset_raw = dataset_raw[dataset_raw["source_ds"] == "pubmedQA" ]

In [3]:
dataset_raw[dataset_raw["label"].isna()]

,id,passage,question,answer,label,source_ds


In [4]:
ds_ex =[dspy.Example({"question":e["question"],"context":e["passage"],"gold_answer":e["answer"],"id":e["id"],"label":e["label"]}).with_inputs("question","context") for e in dataset_raw.to_dict('records')]

In [5]:
class GenerateAnswer(dspy.Signature):
   # """Answer questions with one word answers.""" #used for Finance bench
    """Answer questions with short factoid answers.""" # used for PubMedQA

    context = dspy.InputField(desc="contains relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField()

class RAG(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self,question,context):
        prediction = self.generate_answer(context=context , question = question)
        return dspy.Prediction(context=context,answer= prediction.answer)

In [6]:
len_of_data = 200

In [7]:
from tqdm import tqdm

azure_openai = dspy.AzureOpenAI(api_base=os.getenv("AZURE_OPENAI_ENDPOINT") ,api_key=os.getenv("AZURE_OPENAI_API_KEY"),api_version="2023-05-15",model="chat-demo")
dspy.settings.configure(lm=azure_openai)

rag = RAG()
for e in tqdm(ds_ex[:len_of_data],desc="Processing questions",unit="question"):
    pred = rag(question=e["question"],context=e["context"])
    e["answer"] = pred.answer

Processing questions:   0%|          | 0/200 [00:00<?, ?question/s]WARNING:root:	*** In DSPy 2.5, all LM clients except `dspy.LM` are deprecated, underperform, and are about to be deleted. ***
 		You are using the client AzureOpenAI, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


Processing questions: 100%|██████████| 200/200 [00:00<00:00, 412.82question/s]


### Making the data results for pubmed with GPT3.5

In [8]:
import json
data ={
        'question':[],
        'answer':[],
        'contexts':[],
        'ground_truth':[],
        'label':[],
        'id':[]
    }

for example in ds_ex[:len_of_data]:
    # Append data to the respective lists
    data['question'].append(example.question)
    data['contexts'].append([example.context])  # Note: This is a list with one item
    data['ground_truth'].append(example.gold_answer)
    data['answer'].append(example["answer"])
    data['label'].append(example["label"])
    data['id'].append(example["id"])

# Save the data to a JSON file
with open('results/pubmed_results.json', 'w', encoding='utf-8') as f: #fs_results.json
    json.dump(data, f, ensure_ascii=False, indent=4)